# Initialize the HDFS and the resource manager
Keep in mind these commands will vary depending on where do you have this notebook and where is installed your hadoop.

In [1]:
# Start the dfs
!./hadoop-3.3.6/sbin/start-dfs.sh

Starting namenodes on [localhost]
Starting datanodes
Starting secondary namenodes [debian]


In [2]:
# Start the resource manager (YARN in our case)
!./hadoop-3.3.6/sbin/start-yarn.sh

Starting resourcemanager
Starting nodemanagers


# Imports and configurations

In [3]:
import os
import pandas as pd

# Pyspark modules
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml import Pipeline, Transformer
from pyspark.ml.feature import VectorAssembler, StandardScaler, MinMaxScaler, UnivariateFeatureSelector
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, NaiveBayes, LogisticRegression
from pyspark.ml.classification import OneVsRest, MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

from pprint import pprint

In [4]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("SparkSessionExample") \
    .master("local[*]") \
    .getOrCreate()

24/01/07 13:01:41 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
# Get all configuration options
conf = spark.sparkContext.getConf().getAll()
pprint(conf)

[('spark.executor.id', 'driver'),
 ('spark.driver.host', '10.0.2.15'),
 ('spark.driver.port', '34563'),
 ('spark.app.name', 'PySparkShell'),
 ('spark.app.startTime', '1704650228007'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.driver.extraJavaOptions',
  '-XX:+IgnoreUnrecognizedVMOptions '
  '--add-opens=java.base/java.lang=ALL-UNNAMED '
  '--add-opens=java.base/java.lang.invoke=ALL-UNNAMED '
  '--add-opens=java.base/java.lang.reflect=ALL-UNNAMED '
  '--add-opens=java.base/java.io=ALL-UNNAMED '
  '--add-opens=java.base/java.net=ALL-UNNAMED '
  '--add-opens=java.base/java.nio=ALL-UNNAMED '
  '--add-opens=java.base/java.util=ALL-UNNAMED '
  '--add-opens=java.base/java.util.concurrent=ALL-UNNAMED '
  '--add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED '
  '--add-opens=java.base/jdk.internal.ref=ALL-UNNAMED '
  '--add-opens=java.base/sun.nio.ch=ALL-UNNAMED '
  '--add-opens=java.base/sun.nio.cs=ALL-UNNAMED '
  '--add-opens=java.base/sun.security.action=ALL-UNNAMED '
 

# Read the data

In [6]:
# Spark accepts wildcards when reading data. (We load all partitions at once)
df = spark.read.parquet('hdfs:///user/hadoop/WESAD_parquet/S*.parquet')
print(f'Total number of samples {df.count()}')

Total number of samples 60807600


In [7]:
# Show data schema
df.printSchema()

root
 |-- ACC_1: double (nullable = true)
 |-- ACC_2: double (nullable = true)
 |-- ACC_3: double (nullable = true)
 |-- ECG: double (nullable = true)
 |-- EMG: double (nullable = true)
 |-- EDA: double (nullable = true)
 |-- Temp: float (nullable = true)
 |-- Resp: double (nullable = true)
 |-- label: integer (nullable = true)



In [8]:
# Show some features
df.select("ACC_1", "ACC_2", "ACC_3", 'label').show(10)

+-------------------+--------------------+--------------------+-----+
|              ACC_1|               ACC_2|               ACC_3|label|
+-------------------+--------------------+--------------------+-----+
|0.35099995136260986| -0.1905999779701233|  2.2841999530792236|    0|
| 0.4421999454498291|-0.11419999599456787|    2.22160005569458|    0|
| 0.5321999788284302|-0.05820000171661377|  1.9242000579833984|    0|
| 0.6926000118255615|-9.99987125396728...|  1.4021999835968018|    0|
| 0.9018000364303589| 0.04499995708465576|  0.6477999687194824|    0|
| 1.1470000743865967| 0.06540000438690186|-0.26319998502731323|    0|
| 1.4089999198913574|0.056800007820129395| -1.2230000495910645|    0|
| 1.5896000862121582|0.017199993133544922| -2.0999999046325684|    0|
|  1.661600112915039|  -0.053600013256073| -2.8071999549865723|    0|
| 1.6314001083374023|-0.14539998769760132|  -3.272599935531616|    0|
+-------------------+--------------------+--------------------+-----+
only showing top 10 

We see data was loaded correctly and that we can display it succesfully.

# Exploratory Data Analysis (EDA) and Data Cleaning

In [9]:
stats = df.describe(df.columns[0:5])
stats.show()

+-------+-------------------+--------------------+--------------------+--------------------+--------------------+
|summary|              ACC_1|               ACC_2|               ACC_3|                 ECG|                 EMG|
+-------+-------------------+--------------------+--------------------+--------------------+--------------------+
|  count|           60807600|            60807600|            60807600|            60807600|            60807600|
|   mean| 0.8116865284321901|-0.04424838120623882|-0.25900813261880495|0.001064647998794382|-0.00303824747415...|
| stddev|0.13125288401427787| 0.10388864087113163|  0.3321663786069044| 0.26868806452510596|0.017904851872606677|
|    min| -6.599999904632568|  -6.599999904632568|  -6.599999904632568|                -1.5|                -1.5|
|    max| 2.9814000129699707|  1.6089999675750732|   4.508200168609619|  1.4999542236328125|  1.4643402099609375|
+-------+-------------------+--------------------+--------------------+-----------------

In [10]:
stats = df.describe(df.columns[5:])
stats.show()

+-------+------------------+------------------+-------------------+------------------+
|summary|               EDA|              Temp|               Resp|             label|
+-------+------------------+------------------+-------------------+------------------+
|  count|          60807600|          60807600|           60807600|          60807600|
|   mean|  4.88823725419249|33.904942453534574|0.05424957702482945|1.3347263335504114|
| stddev| 3.531246876893006|1.2169364535428147|  4.099115876402935|1.6283486656155444|
|    min|               0.0|           -273.15|              -50.0|                 0|
|    max|22.410964965820312|         35.778046|    38.800048828125|                 7|
+-------+------------------+------------------+-------------------+------------------+



Notice there are not missing values, this will saves us some time since we won't need to deal with them.

# Data pipeline
Here we define the pipeline followed by the data to be prepared for the training.

As said in the dataset, labels from 5 to 7 should be ignored in this dataset. In addition, we want to classify only 
the classes "baseline", "stress" and "amusement", therefore we will also remove labels 0 and 4.

We will also change the range of labels to avoid problems with some classifiers, like the Multilayer Perceptron.

---

Here we define a custom transformer which filters the dataframe using the designed filters. 
In our case we defined two filters:
- Label != 0
- Label < 4

In this way, we will remove the labels [0, 4, 5, 6, 7]

In [11]:
# Extends Transformer class and override _transform method. We define our filters in the constructor.
class DataCleaner(Transformer):
    
    def __init__(self, column_name=None, mean=None):
        super().__init__()
        self.filters = ["label <> 0", "label < 4"] # Set of filter applied by the transformer
        
    def _transform(self, dff):
        
        # Filter the dataframe for each filter in the set
        for filt in self.filters:
            dff = dff.filter(filt)
            
        # Move the range of labels from [1, 2, 3] to [0, 1, 2]
        dff = dff.withColumn('label', col('label') - 1)
            
        return dff

The ML model accepts dataframes in the shape of [features, label], then we need to assemble all the feature columns in one single column with all the values. Fortunately, there is a transformer that does this in spark, the "VectorAssembler".

Also to avoid problems with some classifiers (like Naive Bayes) we rescale our features to avoid negative values. For this we use the "MinMaxScaler" which, by default, scales into the range [0,1].

In [12]:
feature_cols = ['ACC_1', 'ACC_2', 'ACC_3', 'ECG', 'EMG', 'EDA', 'Temp', 'Resp'] # Define feature columns

dataCleaner = DataCleaner() # Custom data cleaner
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features') # Assembler for the feature columns
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

In [13]:
# Create the pipeline with the stages followed by the data.
pipeline = Pipeline(stages=[dataCleaner, assembler, scaler])

# Fit pipeline model and transform data
df_cleaned = pipeline.fit(df).transform(df)

In [14]:
print(f'Number of samples after removing useless labeled ones: {df_cleaned.count()}')

Number of samples after removing useless labeled ones: 23206404


# Train the different ML models
For this task we have chosen five classification algorithms
1. Decision Tree
2. Random Forest
3. Naive Bayes
4. Logistic Regression
5. Multilayer Perceptron

In [15]:
# Split the data into train and test
train_data, test_data = df_cleaned.randomSplit([0.7, 0.3], seed=42)

In [17]:
# INSTANTIATE MODELS

# Decision Tree
dt = DecisionTreeClassifier(featuresCol='scaledFeatures', labelCol='label')

# Random Forest
rf = RandomForestClassifier(featuresCol='scaledFeatures', labelCol='label', seed=42)

# Naive Bayes
nb = NaiveBayes(featuresCol='scaledFeatures', labelCol='label', smoothing=0.5, modelType="multinomial")

# Logistic Regresssion with One Vs Rest approach
lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True, featuresCol='scaledFeatures', labelCol='label')
ovr = OneVsRest(classifier=lr)

# Multilayer Perceptron
mlp = MultilayerPerceptronClassifier(layers=[8, 16, 32, 3], maxIter=20, 
                                     featuresCol='scaledFeatures', labelCol='label', seed=42)

In [18]:
dt = dt.fit(train_data)

24/01/07 13:07:57 WARN MemoryStore: Not enough space to cache rdd_74_1 in memory! (computed 147.6 MiB so far)
24/01/07 13:07:57 WARN BlockManager: Persisting block rdd_74_1 to disk instead.
24/01/07 13:07:57 WARN MemoryStore: Not enough space to cache rdd_74_0 in memory! (computed 147.6 MiB so far)
24/01/07 13:07:57 WARN BlockManager: Persisting block rdd_74_0 to disk instead.
24/01/07 13:08:01 WARN MemoryStore: Not enough space to cache rdd_74_0 in memory! (computed 157.6 MiB so far)


24/01/07 13:08:16 WARN MemoryStore: Not enough space to cache rdd_74_2 in memory! (computed 1044.8 KiB so far)
24/01/07 13:08:16 WARN BlockManager: Persisting block rdd_74_2 to disk instead.
24/01/07 13:08:18 WARN MemoryStore: Not enough space to cache rdd_74_3 in memory! (computed 147.6 MiB so far)
24/01/07 13:08:18 WARN BlockManager: Persisting block rdd_74_3 to disk instead.
24/01/07 13:08:22 WARN MemoryStore: Not enough space to cache rdd_74_2 in memory! (computed 157.6 MiB so far)


24/01/07 13:08:40 WARN MemoryStore: Not enough space to cache rdd_74_5 in memory! (computed 147.6 MiB so far)
24/01/07 13:08:40 WARN BlockManager: Persisting block rdd_74_5 to disk instead.
24/01/07 13:08:41 WARN MemoryStore: Not enough space to cache rdd_74_4 in memory! (computed 221.4 MiB so far)
24/01/07 13:08:41 WARN BlockManager: Persisting block rdd_74_4 to disk instead.
24/01/07 13:08:44 WARN MemoryStore: Not enough space to cache rdd_74_4 in memory! (computed 157.6 MiB so far)


24/01/07 13:08:54 WARN MemoryStore: Not enough space to cache rdd_74_7 in memory! (computed 65.6 MiB so far)
24/01/07 13:08:54 WARN BlockManager: Persisting block rdd_74_7 to disk instead.
24/01/07 13:08:57 WARN MemoryStore: Not enough space to cache rdd_74_7 in memory! (computed 157.6 MiB so far)


24/01/07 13:09:00 WARN MemoryStore: Not enough space to cache rdd_74_6 in memory! (computed 147.6 MiB so far)
24/01/07 13:09:00 WARN BlockManager: Persisting block rdd_74_6 to disk instead.


...

24/01/07 13:09:29 WARN MemoryStore: Not enough space to cache rdd_74_0 in memory! (computed 43.8 MiB so far)


24/01/07 13:09:29 WARN MemoryStore: Not enough space to cache rdd_74_1 in memory! (computed 99.2 MiB so far)


24/01/07 13:09:31 WARN MemoryStore: Not enough space to cache rdd_74_2 in memory! (computed 99.2 MiB so far)
24/01/07 13:09:32 WARN MemoryStore: Not enough space to cache rdd_74_3 in memory! (computed 99.2 MiB so far)


24/01/07 13:09:34 WARN MemoryStore: Not enough space to cache rdd_74_4 in memory! (computed 99.2 MiB so far)
24/01/07 13:09:35 WARN MemoryStore: Not enough space to cache rdd_74_5 in memory! (computed 99.2 MiB so far)


24/01/07 13:09:37 WARN MemoryStore: Not enough space to cache rdd_74_7 in memory! (computed 99.2 MiB so far)


In [19]:
# Fit models
rf = rf.fit(train_data)

24/01/07 13:13:00 WARN MemoryStore: Not enough space to cache rdd_119_0 in memory! (computed 149.9 MiB so far)
24/01/07 13:13:00 WARN BlockManager: Persisting block rdd_119_0 to disk instead.
24/01/07 13:13:00 WARN MemoryStore: Not enough space to cache rdd_119_1 in memory! (computed 149.9 MiB so far)
24/01/07 13:13:00 WARN BlockManager: Persisting block rdd_119_1 to disk instead.
24/01/07 13:13:08 WARN MemoryStore: Not enough space to cache rdd_119_1 in memory! (computed 149.9 MiB so far)
24/01/07 13:13:08 WARN MemoryStore: Not enough space to cache rdd_119_0 in memory! (computed 236.8 MiB so far)


24/01/07 13:13:27 WARN MemoryStore: Not enough space to cache rdd_119_2 in memory! (computed 149.9 MiB so far)
24/01/07 13:13:27 WARN BlockManager: Persisting block rdd_119_2 to disk instead.
24/01/07 13:13:28 WARN MemoryStore: Not enough space to cache rdd_119_3 in memory! (computed 149.9 MiB so far)
24/01/07 13:13:28 WARN BlockManager: Persisting block rdd_119_3 to disk instead.
24/01/07 13:13:35 WARN MemoryStore: Not enough space to cache rdd_119_3 in memory! (computed 149.9 MiB so far)
24/01/07 13:13:35 WARN MemoryStore: Not enough space to cache rdd_119_2 in memory! (computed 236.8 MiB so far)


24/01/07 13:13:55 WARN MemoryStore: Not enough space to cache rdd_119_4 in memory! (computed 149.9 MiB so far)
24/01/07 13:13:55 WARN BlockManager: Persisting block rdd_119_4 to disk instead.
24/01/07 13:13:57 WARN MemoryStore: Not enough space to cache rdd_119_5 in memory! (computed 236.8 MiB so far)
24/01/07 13:13:57 WARN BlockManager: Persisting block rdd_119_5 to disk instead.
24/01/07 13:14:04 WARN MemoryStore: Not enough space to cache rdd_119_4 in memory! (computed 236.8 MiB so far)
24/01/07 13:14:04 WARN MemoryStore: Not enough space to cache rdd_119_5 in memory! (computed 149.9 MiB so far)


24/01/07 13:14:17 WARN MemoryStore: Not enough space to cache rdd_119_7 in memory! (computed 99.9 MiB so far)
24/01/07 13:14:17 WARN BlockManager: Persisting block rdd_119_7 to disk instead.
24/01/07 13:14:22 WARN MemoryStore: Not enough space to cache rdd_119_7 in memory! (computed 236.8 MiB so far)
24/01/07 13:14:22 WARN MemoryStore: Not enough space to cache rdd_119_6 in memory! (computed 99.9 MiB so far)
24/01/07 13:14:22 WARN BlockManager: Persisting block rdd_119_6 to disk instead.


24/01/07 13:14:30 WARN MemoryStore: Not enough space to cache rdd_119_6 in memory! (computed 355.2 MiB so far)


...

24/01/07 13:16:12 WARN MemoryStore: Not enough space to cache rdd_119_7 in memory! (computed 149.9 MiB so far)
24/01/07 13:16:13 WARN MemoryStore: Not enough space to cache rdd_119_6 in memory! (computed 236.8 MiB so far)


24/01/07 13:16:21 WARN MemoryStore: Not enough space to cache rdd_119_1 in memory! (computed 236.8 MiB so far)
24/01/07 13:16:21 WARN MemoryStore: Not enough space to cache rdd_119_0 in memory! (computed 149.9 MiB so far)


24/01/07 13:16:29 WARN MemoryStore: Not enough space to cache rdd_119_3 in memory! (computed 149.9 MiB so far)
24/01/07 13:16:30 WARN MemoryStore: Not enough space to cache rdd_119_2 in memory! (computed 236.8 MiB so far)


24/01/07 13:16:39 WARN MemoryStore: Not enough space to cache rdd_119_5 in memory! (computed 66.6 MiB so far)
24/01/07 13:16:39 WARN MemoryStore: Not enough space to cache rdd_119_4 in memory! (computed 355.2 MiB so far)


24/01/07 13:16:48 WARN MemoryStore: Not enough space to cache rdd_119_7 in memory! (computed 66.6 MiB so far)
24/01/07 13:16:49 WARN MemoryStore: Not enough space to cache rdd_119_6 in memory! (computed 355.2 MiB so far)


In [20]:
nb = nb.fit(train_data)

In [21]:
ovr = ovr.fit(train_data)

24/01/07 13:20:40 WARN MemoryStore: Not enough space to cache rdd_182_3 in memory! (computed 17.0 MiB so far)
24/01/07 13:20:40 WARN BlockManager: Persisting block rdd_182_3 to disk instead.
24/01/07 13:20:40 WARN MemoryStore: Not enough space to cache rdd_182_2 in memory! (computed 65.0 MiB so far)
24/01/07 13:20:41 WARN BlockManager: Persisting block rdd_182_2 to disk instead.
24/01/07 13:20:45 WARN MemoryStore: Not enough space to cache rdd_182_2 in memory! (computed 113.0 MiB so far)


24/01/07 13:20:47 WARN MemoryStore: Not enough space to cache rdd_182_3 in memory! (computed 113.0 MiB so far)


24/01/07 13:21:04 WARN MemoryStore: Not enough space to cache rdd_182_4 in memory! (computed 113.0 MiB so far)
24/01/07 13:21:04 WARN BlockManager: Persisting block rdd_182_4 to disk instead.
24/01/07 13:21:05 WARN MemoryStore: Not enough space to cache rdd_182_5 in memory! (computed 65.0 MiB so far)
24/01/07 13:21:05 WARN BlockManager: Persisting block rdd_182_5 to disk instead.


24/01/07 13:21:08 WARN MemoryStore: Not enough space to cache rdd_182_5 in memory! (computed 33.0 MiB so far)


24/01/07 13:21:21 WARN MemoryStore: Not enough space to cache rdd_182_6 in memory! (computed 65.0 MiB so far)
24/01/07 13:21:21 WARN BlockManager: Persisting block rdd_182_6 to disk instead.


...

24/01/07 13:27:08 WARN MemoryStore: Not enough space to cache rdd_350_5 in memory! (computed 65.0 MiB so far)


24/01/07 13:27:08 WARN MemoryStore: Not enough space to cache rdd_350_0 in memory! (computed 33.0 MiB so far)
24/01/07 13:27:08 WARN MemoryStore: Not enough space to cache rdd_350_1 in memory! (computed 33.0 MiB so far)
24/01/07 13:27:09 WARN MemoryStore: Not enough space to cache rdd_350_2 in memory! (computed 65.0 MiB so far)
24/01/07 13:27:09 WARN MemoryStore: Not enough space to cache rdd_350_3 in memory! (computed 17.0 MiB so far)


24/01/07 13:27:09 WARN MemoryStore: Not enough space to cache rdd_350_5 in memory! (computed 65.0 MiB so far)


24/01/07 13:27:09 WARN MemoryStore: Not enough space to cache rdd_350_1 in memory! (computed 33.0 MiB so far)
24/01/07 13:27:09 WARN MemoryStore: Not enough space to cache rdd_350_0 in memory! (computed 33.0 MiB so far)
24/01/07 13:27:10 WARN MemoryStore: Not enough space to cache rdd_350_2 in memory! (computed 33.0 MiB so far)
24/01/07 13:27:10 WARN MemoryStore: Not enough space to cache rdd_350_3 in memory! (computed 33.0 MiB so far)


24/01/07 13:27:10 WARN MemoryStore: Not enough space to cache rdd_350_5 in memory! (computed 65.0 MiB so far)


In [22]:
mlp = mlp.fit(train_data)

24/01/07 13:27:30 WARN MemoryStore: Not enough space to cache rdd_432_0 in memory! (computed 166.2 MiB so far)
24/01/07 13:27:30 WARN BlockManager: Persisting block rdd_432_0 to disk instead.
24/01/07 13:27:30 WARN MemoryStore: Not enough space to cache rdd_432_1 in memory! (computed 166.2 MiB so far)
24/01/07 13:27:30 WARN BlockManager: Persisting block rdd_432_1 to disk instead.


24/01/07 13:27:46 WARN MemoryStore: Not enough space to cache rdd_432_3 in memory! (computed 48.2 MiB so far)
24/01/07 13:27:46 WARN BlockManager: Persisting block rdd_432_3 to disk instead.
24/01/07 13:27:47 WARN MemoryStore: Not enough space to cache rdd_432_2 in memory! (computed 110.7 MiB so far)
24/01/07 13:27:47 WARN BlockManager: Persisting block rdd_432_2 to disk instead.
24/01/07 13:27:50 WARN MemoryStore: Not enough space to cache rdd_432_2 in memory! (computed 166.2 MiB so far)


24/01/07 13:27:51 WARN MemoryStore: Not enough space to cache rdd_432_3 in memory! (computed 166.2 MiB so far)


24/01/07 13:28:04 WARN MemoryStore: Not enough space to cache rdd_432_4 in memory! (computed 9.1 MiB so far)
24/01/07 13:28:04 WARN BlockManager: Persisting block rdd_432_4 to disk instead.
24/01/07 13:28:08 WARN MemoryStore: Not enough space to cache rdd_432_5 in memory! (computed 110.7 MiB so far)
24/01/07 13:28:08 WARN BlockManager: Persisting block rdd_432_5 to disk instead.
24/01/07 13:28:10 WARN MemoryStore: Not enough space to cache rdd_432_4 in memory! (computed 166.2 MiB so far)


24/01/07 13:28:11 WARN MemoryStore: Not enough space to cache rdd_432_5 in memory! (computed 166.2 MiB so far)


...

24/01/07 13:36:56 WARN MemoryStore: Not enough space to cache rdd_432_2 in memory! (computed 110.7 MiB so far)
24/01/07 13:36:56 WARN MemoryStore: Not enough space to cache rdd_432_3 in memory! (computed 48.2 MiB so far)


24/01/07 13:37:02 WARN MemoryStore: Not enough space to cache rdd_432_4 in memory! (computed 73.7 MiB so far)
24/01/07 13:37:02 WARN MemoryStore: Not enough space to cache rdd_432_5 in memory! (computed 73.7 MiB so far)


24/01/07 13:37:14 WARN MemoryStore: Not enough space to cache rdd_432_1 in memory! (computed 48.2 MiB so far)
24/01/07 13:37:14 WARN MemoryStore: Not enough space to cache rdd_432_0 in memory! (computed 110.7 MiB so far)


24/01/07 13:37:20 WARN MemoryStore: Not enough space to cache rdd_432_2 in memory! (computed 110.7 MiB so far)
24/01/07 13:37:20 WARN MemoryStore: Not enough space to cache rdd_432_3 in memory! (computed 48.2 MiB so far)


24/01/07 13:37:26 WARN MemoryStore: Not enough space to cache rdd_432_4 in memory! (computed 110.7 MiB so far)
24/01/07 13:37:26 WARN MemoryStore: Not enough space to cache rdd_432_5 in memory! (computed 48.2 MiB so far)


# Test ML models

In [23]:
# Get the predictions from all the models for the test data
predictions_dt = dt.transform(test_data)
predictions_rf = rf.transform(test_data)
predictions_nb = nb.transform(test_data)
predictions_ovr = ovr.transform(test_data)
predictions_mlp = mlp.transform(test_data)

# List with all predictions
predictions = [
    predictions_dt,
    predictions_rf,
    predictions_nb,
    predictions_ovr,
    predictions_mlp
]

In [24]:
# Create the evaluator
evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')

# Get the evaluation of the metric for each prediction made by all the classifiers
accuracy_scores = [evaluator.evaluate(prediction) for prediction in predictions]

# Dictionary with the results for each classifier (helps on display later)
results_dict = {
    "Classifier": ["Decision Tree", 'Random Forest', 'Naive Bayes', 'Logistic Regression', 'Multilayer Perceptron'],
    "Accuracy": accuracy_scores
}

In [27]:
results_df = pd.DataFrame(results_dict)
results_df

,Classifier,Accuracy
0,Decision Tree,0.762597
1,Random Forest,0.801740
2,Naive Bayes,0.531165
3,Logistic Regression,0.553291
4,Multilayer Perceptron,0.553286
